# Interferogram Processing with ISCE topsApp

In [ ]:
#Skip if environment is already created. 
#!mamba env create -f environment.yml

In [1]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.plot
from rasterio import logging
from osgeo import gdal

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-08-08 23:42:17,834 - matplotlib - DEBUG - (private) matplotlib data path: /srv/conda/envs/isce2/lib/python3.6/site-packages/matplotlib/mpl-data
2022-08-08 23:42:17,836 - matplotlib - DEBUG - matplotlib data path: /srv/conda/envs/isce2/lib/python3.6/site-packages/matplotlib/mpl-data
2022-08-08 23:42:17,842 - matplotlib - DEBUG - CONFIGDIR=/home/jovyan/.config/matplotlib
2022-08-08 23:42:17,845 - matplotlib - DEBUG - matplotlib version 3.3.4
2022-08-08 23:42:17,846 - matplotlib - DEBUG - interactive is False
2022-08-08 23:42:17,848 - matplotlib - DEBUG - platform is lin

In [18]:
os.chdir('/home/jovyan/rmnp_landslide')

In [19]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

# Set path to topsStack workflow in ISCE source directory
CWD = os.getcwd()
os.environ['PATH']+=f':{CWD}/tools/isce2/contrib/stack/topsStack'
print(os.environ['PATH'])

/srv/conda/envs/isce2/bin:/srv/conda/condabin:/srv/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/applications/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/applications:~/isce2/contrib/stack/topsStack/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/applications:/tmp/orbital/tools/isce2/contrib/stack/topsStack/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/applications:/home/jovyan/rmnp_landslide/tools/isce2/contrib/stack/topsStack


In [20]:
!which topsApp.py

/srv/conda/envs/isce2/lib/python3.6/site-packages/isce/applications/topsApp.py


In [4]:
# make required directories
dirs = ['/tmp/orbital', '/tmp/SLC', '/tmp/DEM', '/tmp/AUX']
for i in dirs:
    if not os.path.exists(i):
        os.makedirs(i)

listdir('/tmp')

['orbital', 'DEM', 'AUX', 'SLC', 'tmpmbm3kp2r']

In [5]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

 ········


## Download SLCs

In [27]:
%%time

import asf_search

# Change to SLC directory 
os.chdir('/tmp/SLC')

logging.getLogger("urllib3").setLevel(logging.WARNING)

reference = 'S1B_IW_SLC__1SDV_20210611T010212_20210611T010239_027302_0342C7_3396'
secondary = 'S1B_IW_SLC__1SDV_20210623T010213_20210623T010240_027477_0347EE_D669'
granules = [reference, secondary]

#results = asf_search.granule_search(granules)
#session = asf_search.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
#results.download(path='/tmp/SLC', processes=2, session=session)

CPU times: user 238 µs, sys: 146 µs, total: 384 µs
Wall time: 320 µs


In [24]:
# Create a list of zipped SLCs. Assume there are no other files with .zip extension in SLC directory
tmp = listdir('/tmp/SLC')

scenes = []
for file in tmp:
    if file[-4:]=='.zip':
        scenes.append(file[:-4])
        
print(scenes)

['S1B_IW_SLC__1SDV_20210611T010212_20210611T010239_027302_0342C7_3396', 'S1B_IW_SLC__1SDV_20210623T010213_20210623T010240_027477_0347EE_D669']


## Get orbital files

In [21]:
os.chdir('/tmp/orbital')

In [22]:
!which fetchOrbit.py

/home/jovyan/rmnp_landslide/tools/isce2/contrib/stack/topsStack/fetchOrbit.py


In [ ]:
# grab orbital files with fetchOrbit.py
for file in scenes:
    os.system(f'fetchOrbit.py -i {file}')

## Check DEM
If using your own DEM, place it in the 'DEM' directory. Must be in WGS 84. Name your DEM 'demLat.......wgs84
If you don't provide a DEM, dem.py can be used to automatically download SRTM. 

## Write input file

In [43]:
os.chdir('/tmp')

In [66]:
with open('topsApp.xml', 'w') as f:
    f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
        <property name="orbit directory">/tmp/orbital</property>
        <property name="output directory">reference</property>
        <property name="safe">/tmp/SLC/{reference}.zip</property>
    </component>
    <component name="secondary">
        <property name="orbit directory">/tmp/orbital</property>
        <property name="output directory">secondary</property>
        <property name="safe">/tmp/SLC/{secondary}.zip</property>
    </component>
    <property name="demfilename">/tmp/DEM/demLat_3dep_1m.dem.wgs84</property>
    <property name="useGPU">False</property>
    <property name="swaths">[1]</property>
    <property name="range looks">3</property>
    <property name="azimuth looks">1</property>
    <property name="region of interest">[40.23, 40.50, -105.69, -105.67]</property>
    <property name="do esd">False</property>
    <property name="do ionospherecorrection">False</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <property name="do denseoffsets">False</property>
    <property name="geocode demfilename">path_to_your_dem</property>
    <!--<property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property> -->
  </component>
</topsApp>""")

## topsApp processing

In [67]:
!topsApp.py --dostep=preprocess

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-08-09 00:30:07,528 - isce.insar - INFO - ISCE VERSION = 2.5.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210305, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210305, CURRENT_SVN_REVISION = 
Step processing
Cannot open PICKLE/startup
Running step preprocess
Input XML files:  ['/vsizip//tmp/SLC/S1B_IW_SLC__1SDV_20210611T010212_20210611T010239_027302_0342C7_3396.zip/S1B_IW_SLC__1SDV_20210611T010212_20210611T010239_027302_0342C7_3396.SAFE/annotation/s1b-iw1-slc-vv-20210611t010213-20210611t010238-027302-0342c7-004.xml']
Input TIFF f

In [56]:
!topsApp.py --dostep=computeBaselines

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-08-09 00:16:36,617 - isce.insar - INFO - ISCE VERSION = 2.5.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210305, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210305, CURRENT_SVN_REVISION = 
Step processing
Running step computeBaselines
Skipping processing for swath number IW-1
No swaths contain any burst overlaps ... cannot continue for interferometry applications
2022-08-09 00:16:36,668 - isce.topsinsar.runPreprocessor - INFO - 
###################################################################################################

In [ ]:
!topsApp.py --dostep=verifyDEM

In [ ]:
!topsApp.py --dostep=topo

In [ ]:
!topsApp.py --start=subsetoverlaps --end=mergebursts

In [ ]:
!topsApp.py --start=filter --end=geocode